In [566]:
# 라이브러리 가져오기
import numpy as np
import pandas as pd

# 원본 데이터 가져오기
기본정보_raw = pd.read_csv('기본 정보(원본).csv', encoding = 'utf-8')
운영정보_raw = pd.read_csv('운영 정보(원본).csv', encoding = 'utf-8')
메뉴정보_raw = pd.read_csv('메뉴 정보(원본).csv', encoding = 'utf-8')

# 필요한 컬럼만 선택하기
# 도로명 주소는 상세 정보에 사용하고, 지번 주소는 전처리에 사용함
# 장동, 궁동, 대의동, 금남로 1~3가, 충장로 1~3가, 대인동, 동명동, 서석동, 광산동
기본정보 = 기본정보_raw[['식당ID', '식당명', '도로명주소', '지번주소', '식당위도', '식당경도', '식당전화번호', '식당업태']].copy()
운영정보 = 운영정보_raw[['식당ID', '휴무일정보', '영업시간']].copy()

# 기본정보 테이블, 운영정보 테이블 병합하기
기본정보 = pd.merge(기본정보, 운영정보, on = '식당ID')
기본정보 = 기본정보[['식당ID', '식당명', '도로명주소', '지번주소', '식당업태', '식당위도', '식당경도', '식당전화번호', '영업시간', '휴무일정보']].copy()

# 업태 정리하기(한식, 중식, 일식, 양식, 분식, 뷔페, 식육, 외국음식, 패스트푸드)
for i in range(0, 기본정보.shape[0], 1) :
    if 기본정보['식당업태'][i] == '경양식' :
        기본정보['식당업태'][i] = '양식'
    elif 기본정보['식당업태'][i] == '김밥(도시락)' :
        기본정보['식당업태'][i] = '분식'
    elif 기본정보['식당업태'][i] == '냉면집' :
        기본정보['식당업태'][i] = '한식'
    elif 기본정보['식당업태'][i] == '뷔페식' :
        기본정보['식당업태'][i] = '뷔페'
    elif 기본정보['식당업태'][i] == '식육(숯불구이)' :
        기본정보['식당업태'][i] = '식육'
    elif 기본정보['식당업태'][i] == '외국음식전문점(인도,태국등)' :
        기본정보['식당업태'][i] = '외국음식'
    elif 기본정보['식당업태'][i] == '중국식' :
        기본정보['식당업태'][i] = '중식'
    elif 기본정보['식당업태'][i] == '탕류(보신용)' :
        기본정보['식당업태'][i] = '한식'
    elif 기본정보['식당업태'][i] == '패밀리레스토랑' :
        기본정보['식당업태'][i] = '양식'
    elif 기본정보['식당업태'][i] == '회집' :
        기본정보['식당업태'][i] = '일식'
        
# 필요 없는 업태 삭제하기
기본정보 = 기본정보[(기본정보['식당업태'] != '기타')][(기본정보['식당업태'] != '커피숍')][(기본정보['식당업태'] != '호프/통닭')][(기본정보['식당업태'] != '기타 휴게음식점')][(기본정보['식당업태'] != '제과점영업')][(기본정보['식당업태'] != '정종/대포집/소주방')][(기본정보['식당업태'] != '다방')][(기본정보['식당업태'] != '일반조리판매')][(기본정보['식당업태'] != '기타(편의점)')][(기본정보['식당업태'] != '푸드트럭')][(기본정보['식당업태'] != '전통찻집')][(기본정보['식당업태'] != '백화점')][(기본정보['식당업태'] != '라이브카페')][(기본정보['식당업태'] != '아이스크림')][(기본정보['식당업태'] != '철도역구내')].copy()

# 업태 결측치 제거하기
기본정보 = 기본정보[기본정보['식당업태'].notnull()]

# 주소 결측치 제거하기
기본정보['도로명주소'].isnull().value_counts() # 총 1371개, 결측치 5개, 나머지 1366개
기본정보['지번주소'].isnull().value_counts() # 총 1371개, 결측치 5개, 나머지 1366개
기본정보 = 기본정보[기본정보['도로명주소'].notnull()]
기본정보 = 기본정보[기본정보['지번주소'].notnull()]

# 인덱스 다시 지정하기
기본정보.reset_index(inplace = True)
기본정보 = 기본정보[['식당ID', '식당명', '도로명주소', '지번주소', '식당업태', '식당위도', '식당경도', '식당전화번호', '영업시간', '휴무일정보']].copy()

# 필요 없는 구역 제거하기(지번 주소로부터 제거하기)
# 장동, 궁동, 대의동, 금남로 1~3가, 충장로 1~3가, 대인동, 동명동, 서석동, 광산동
동목록 = ['장동', '궁동', '대의동', '금남로1가', '금남로2가', '금남로3가',
      '충장로1가', '충장로2가', '충장로3가', '대인동', '동명동', '서석동', '광산동']
동체크 = []
for i in range(0, 기본정보.shape[0], 1) :
    if 기본정보['지번주소'][i].split()[2] in 동목록 :
        동체크.append(True)
    else :
        동체크.append(False)
        
기본정보 = 기본정보[동체크]
기본정보.reset_index(inplace = True)
기본정보 = 기본정보[['식당ID', '식당명', '도로명주소', '지번주소', '식당업태', '식당위도', '식당경도', '식당전화번호', '영업시간', '휴무일정보']].copy()



In [578]:
# 개발원까지의 거리 계산해서 걸러내기
from haversine import haversine
import math

# 식당의 위도, 경도를 튜플 형태로 리스트에 저장함
식당위치 = []
for i in range(0, 기본정보.shape[0], 1) :
    식당좌표 = (기본정보['식당위도'][i], 기본정보['식당경도'][i])
    식당위치.append(식당좌표)
    
식당거리 = []
개발원위치 = (35.15009633538209, 126.91979751121468)
for i in range(0, 기본정보.shape[0], 1) :
    식당거리.append(round(haversine(개발원위치, 식당위치[i], unit = 'm')))
    
기본정보.loc[ : , '식당거리'] = 식당거리
기본정보 = 기본정보[['식당ID', '식당명', '도로명주소', '식당업태', '식당거리', '식당전화번호', '영업시간', '휴무일정보']]

기본정보 = 기본정보[기본정보['식당거리'] < 500]
기본정보.reset_index(inplace = True)
기본정보 = 기본정보[['식당ID', '식당명', '도로명주소', '식당업태', '식당거리', '식당전화번호', '영업시간', '휴무일정보']]

# 기본정보.to_csv('기본 정보(수정).csv', encoding = 'euc-kr')

In [ ]:
# 기본정보, 메뉴정보 파일 정리하기

# 기본정보
# 컬럼명 다시 한 번 확인하기

# 메뉴정보
# 점심 메뉴로 적절하지 않은 메뉴(음료류, 주류 등) 제거하기
# 토핑, 재료 추가, 후식류들 걸러내기
# 사이즈가 다른 메뉴들 하나로 통합하기
# 용어 맞춤법 통일하기(페퍼로니, 페페로니 등)
# 소분류, 자체분류 전부 확인하기